In [1]:
import json
import xml.etree.ElementTree as ET
import sys
import codecs
from tqdm import tqdm
import os
import pandas as pd
import pickle as pk 

In [2]:
path_to_utils = '../../utils'
path_to_utils = 'SQuAD-task/utils/'
sys.path.append(path_to_utils)
from data_utils import Vocabulary, Token_Node

In [3]:
vocab = Vocabulary()

In [5]:
def read_data(path_to_original_data = './Original/Train.txt'):
    print 'Reading Data ...'
    with open(path_to_original_data,'r') as f:
        DataJ = f.read()
    data = json.loads(DataJ)
    # vocab.make_vocab(data)
    for d in tqdm(data['data'][:2]):
        for p in d['paragraphs']:
            context = p['context']
            for q in p['qas']:
                for a in q['answers']:
                    answer = a['text']
                    answer_start, answer_end = int(a['answer_start']), int(a['answer_start'])+len(answer)
                    assert answer == context[answer_start:answer_end], "{} {} {}".format(a['text'], answer_start, answer_end)
                    a['answer_end'] = answer_end
    return data

In [6]:
def coreNLP(text,is_context = True):
    print "\n****  Parsing >>> ",text,' ****\n'
    f = codecs.open('./input.txt','w','utf-8-sig')
    f.write(text)
    f.close()
    os.system('java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit -file input.txt')
    tree = ET.parse('input.txt.xml')
    root = tree.getroot()
    sentences = root.find('document').find('sentences').findall('sentence')
    sentence_list = []
    w_index = 0
    for s in sentences:
        sent = []
        # Get the Token informations from the pipeline
        for tok in s.find('tokens').findall('token'):
            if is_context:
                char_begin, char_end = int(tok.find('CharacterOffsetBegin').text) -1, int(tok.find('CharacterOffsetEnd').text) -1
                Token = Token_Node( w_index, tok.find('word').text, char_begin, char_end)
                # to be indexed from the start of the para.
            else:
                tok_id = int(tok.attrib['id']) - 1
                Token = Token_Node( tok_id, tok.find('word').text )
            sent.append(Token)
            w_index += 1
        sentence_list.append(sent)
    return sentence_list

In [7]:
def augment_data(data):
    os.chdir('./StanfordCoreNLP/')
    error_count, error_list = 0, []
    for d in tqdm(data['data'][:2]):
        for p in d['paragraphs']:
            aug_context = coreNLP(p['context'], True)
            words = []
            for i,s in enumerate(aug_context):
                words = words + s
                for w in s:
                    if not vocab.word_to_index.has_key(w.word.lower()): 
                        vocab.insert(w.word.lower())
            p['words'] = words
            p['tokens'] = [ w.word for w in words ] #[ vocab.get_word_index(w.word) for w in words ]

            # Now augment the question
            for q in p['qas']:
                aug_question = coreNLP(q['question'], False)
                q['tokens'] = [ w.word for w in aug_question[0] ]# [ vocab.get_word_index(w.word) for w in aug_question[0] ]
                for w in aug_question[0]:
                    if not vocab.word_to_index.has_key(w.word.lower()): 
                        vocab.insert(w.word.lower())

            # Now, time to augment the answers
            for q in p['qas']:
                print q['tokens']
                for a in q['answers']:
                    print a['text']
                    print a['answer_start']
                    print a['answer_end']
                    answer_start, answer_end = a['answer_start'], a['answer_end']
                    start, end = 0,len(p['tokens'])
                    for w in p['words']: 
                        if w.char_begin == answer_start:
                            start = w.id
                        if w.char_end == answer_end:
                            end = w.id
                    if start > end :
                        error_message = d['title']+'\nPARA:\n'+p['context']+'\nQuestion:\n'+q['question']+'\nAnswer:\n'+a['text']+'\n(start > end): '+start + '>' ,end
                        print '******** ERROR in *********\n',error_message
                        error_list.append()
                        error_count += 1
                    a['begin_id'], a['end_id'] = start, end
                q.pop('question')
            p.pop('context')
            p.pop('words')
    print "No. of ERRORs in Answers:",error_count
    for e in error_list:
        print '\n\n\n\n******** ERROR in *********'
        print e
    os.chdir('../')


In [8]:
def write_out(data, target_path = 'PreProcessed_Data/augmented_train.txt', file_type = None):
    if file_type == 'csv':
        feature_list = ["id","context","question","answer_start","answer_end"]
        df = pd.DataFrame(columns=feature_list)
        i = 0
        for d in tqdm(data['data'][:2]):
            for p in d['paragraphs']:
                for q in p['qas']:
                    for a in q['answers']:
                        df.loc[i] = [ i,p['tokens'],q['tokens'],a['begin_id'], a['end_id'] ]
                        i += 1
        df.to_csv("Data.csv",index=None, columns = feature_list)
        return
    with open(target_path,'w') as f:
        json.dump(data,f)



In [9]:
if __name__ == '__main__':
    if False:
        data = None
        # Read Data
        if len(sys.argv)>1:
            data = read_data(sys.argv[1]) 
        else:
            data = read_data()
        # Augment Data
        augment_data(data)
        # Writeout Data
        if len(sys.argv)>2:
            write_out(sys.argv[2],file_type = 'csv') 
        else:
            write_out(file_type = 'csv')
    else:
        data = None
        # Read Data
        data = read_data()
        # Augment Data
        augment_data(data)
        # Writeout Data
        write_out(data, file_type = 'csv')
        with open('pickled-vocab','wb') as f:
            pk.dump(vocab,f,2)

Reading Data ...


  0%|          | 0/2 [00:00<?, ?it/s]


****  Parsing >>>  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.  ****


****  Parsing >>>  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?  ****


****  Parsing >>>  What is in front of the Notre Dame Main Building?  ****


****  Parsing >>>  The Basilica of the Sacred heart at Notre Dame is beside to which structure?  ***


****  Parsing >>>  Which organization declared the First Year of Studies program at Notre Dame "outstanding?"  ****

['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
Learning Resource Center
496
520
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
five
68
72
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
The First Year of Studies program
155
188
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
U.S. News & World Report
647
671

****  Parsing >>>  The university first offered graduate degrees, in the form of a Master of Arts (MA), in the 1854–1855 academic year. The program expanded to include Master of Laws (LL.M.) and Master of Civil Engineering in its early stages of growth, before a for


****  Parsing >>>  What percentage of Notre Dame students decide to study abroad?  ****

['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
18th overall
32
44
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
8th
362
365
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
1st overall
565
576
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
USA Today
155
164
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
57.6%
918
923

****  Parsing >>>  Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and Physics until 1874. Carrier taught that scientific research and its promise

...


****  Parsing >>>  What was the name of the game put out by Starwave Mobile in 2008 that featured Beyoncé fashions?  ****


****  Parsing >>>  What was the name of the junior fashions launched in 2009 by Beyoncé and her mother?  ****


****  Parsing >>>  Who did Beyoncé team up with in 2010 to get her fashions into Brazil?  ****


****  Parsing >>>  Sasha Fierce for Deréon fashions were sold at stores that included Macy's and what other store?  ****

['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
shoe
51
55
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
Brazil
741
747
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
2009
294
298
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
House of 


****  Parsing >>>  What did she participate in with George Clooney?  ****


****  Parsing >>>  Which two stars did Beyoncé help with their Haiti Earthquake organization?  ****


****  Parsing >>>  What did Beyoncé open at the Brooklyn Phoenix House in 2010?  ****


****  Parsing >>>  What Lee Greenwood song did Beyoncé cover after Osama bin Laden was killed?  ****

['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
George Clooney and Wyclef Jean
24
54
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
Brooklyn Phoenix House
356
378
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
God Bless the USA
781
798
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
$1 million
253
263
['How', 'many', 'miles

  0%|          | 0/2 [00:00<?, ?it/s]


****  Parsing >>>  What song did Beyoncé donate to the 2012 World Humanitarian Day campaign?  ****


****  Parsing >>>  Who did Beyoncé work with in 2013 on the Chime for Change campaign?  ****

['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
Sandy Hook Elementary School shooting
280
317
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
I Was Here
412
422
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
Miss a Meal
1553
1564
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
London
765
771
['How', 'many', 'miles', 'does', 'the', 'average', 'student', 'at', 'Notre', 'Dame', 'travel', 'to', 'study', 'there', '?']
her mother
1277
1287
['How', 'many', 'miles', 'does', 'the', 'average', 'student',

100%|██████████| 2/2 [00:02<00:00,  1.11s/it]
